In [1]:
import sys
import operator

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1589862977500_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from pyspark.sql.window import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
airport_data = "s3://air-traffic-dataset/ontimeperformance_airports.csv"
aircraft_data = "s3://air-traffic-dataset/ontimeperformance_aircrafts.csv"
airlines_data = "s3://air-traffic-dataset/ontimeperformance_airlines.csv"

flights_tiny = "s3://air-traffic-dataset/ontimeperformance_flights_tiny.csv"
flights_small = "s3://air-traffic-dataset/ontimeperformance_flights_small.csv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
airlines_df = spark.read.csv(airlines_data, header=True).cache()
aircraft_df = spark.read.csv(aircraft_data, header=True).cache()
airport_df = spark.read.csv(airport_data, header=True).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Task 1

In [5]:
flights_tiny_df = spark.read.csv(flights_tiny, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
cessna_models = aircraft_df \
    .filter(aircraft_df["manufacturer"] == "CESSNA") \
    .withColumn("modelName", F.regexp_extract(F.col("model"), 
                                 '\d{3}', 0))
cessna_models \
    .select("manufacturer", "model", "modelName", "tailnum") \
    .cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[manufacturer: string, model: string, modelName: string, tailnum: string]

In [7]:
cessna_models.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+------------+----------+----------+------+--------------------+-------------+----+---------+
|tailnum|       type|manufacturer|issue_date|     model|status|       aircraft_type|  engine_type|year|modelName|
+-------+-----------+------------+----------+----------+------+--------------------+-------------+----+---------+
| N201AA|   Co-Owner|      CESSNA|04/14/1992|       150| Valid|Fixed Wing Single...|Reciprocating|1959|      150|
| N202AA|Corporation|      CESSNA|02/12/1996|      421C| Valid|Fixed Wing Multi-...|Reciprocating|1980|      421|
|   N293|Corporation|      CESSNA|04/18/2007|     T337G| Valid|Fixed Wing Multi-...|Reciprocating|1975|      337|
| N3744D| Individual|      CESSNA|07/29/1987|      182A| Valid|Fixed Wing Single...|Reciprocating|1957|      182|
| N378AA| Individual|      CESSNA|06/06/2008|      172E| Valid|Fixed Wing Single...|Reciprocating|1963|      172|
| N421AA|Corporation|      CESSNA|10/24/2007|      421C| Valid|Fixed Wing Multi-...|Reci

In [8]:
flights_count = flights_tiny_df.groupby("tail_number") \
    .count() \
    .cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
cessna_results = cessna_models.join(flights_count, cessna_models.tailnum == flights_count.tail_number, how="left") \
    .groupby("modelName") \
    .sum("count") \
    .sort("sum(count)", ascending=False).cache()
cessna_results.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------+
|modelName|sum(count)|
+---------+----------+
|      421|       535|
|      210|       474|
|      172|       399|
|      182|       350|
|      150|       254|
|      550|       248|
|      337|         8|
+---------+----------+

In [10]:
results = cessna_results.collect()
results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(modelName='421', sum(count)=535), Row(modelName='210', sum(count)=474), Row(modelName='172', sum(count)=399), Row(modelName='182', sum(count)=350), Row(modelName='150', sum(count)=254), Row(modelName='550', sum(count)=248), Row(modelName='337', sum(count)=8)]

In [11]:
def printCessnaModels(cessna_results):
    results = cessna_results.collect()
    results.sort(key=operator.itemgetter(1), reverse=True)
    for i in range(3):
        print("Cessna {} \t {}".format(
            results[i][0], 
            results[i][1]),
            file=sys.stdout
        )
        
printCessnaModels(cessna_results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cessna 421 	 535
Cessna 210 	 474
Cessna 172 	 399

# Task 2

In [12]:
flights_tiny_df = spark.read.csv(flights_tiny, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
flight_cols = [
    "carrier_code",
    "tail_number",
    "delayed_time"
]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
flights_tiny_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- flight_id: string (nullable = true)
 |-- carrier_code: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_date: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- tail_number: string (nullable = true)
 |-- scheduled_depature_time: string (nullable = true)
 |-- scheduled_arrival_time: string (nullable = true)
 |-- actual_departure_time: string (nullable = true)
 |-- actual_arrival_time: string (nullable = true)
 |-- distance: string (nullable = true)

In [15]:
flights_tiny_df \
    .filter(F.col("actual_departure_time") == "24:00:00") \
    .select("flight_id", "carrier_code", "scheduled_depature_time", "actual_departure_time") \
    .show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------+-----------------------+---------------------+
|flight_id|carrier_code|scheduled_depature_time|actual_departure_time|
+---------+------------+-----------------------+---------------------+
|  5301680|          US|               23:55:00|             24:00:00|
|  5437149|          UA|               23:55:00|             24:00:00|
|  6170238|          US|               23:45:00|             24:00:00|
|  7274936|          DL|               22:45:00|             24:00:00|
|  8144810|          DL|               23:50:00|             24:00:00|
+---------+------------+-----------------------+---------------------+
only showing top 5 rows

In [16]:
YEAR = 1996

flights_tiny_df = flights_tiny_df \
    .filter(flights_tiny_df["flight_date"].between(
        "{}-01-01".format(YEAR), 
        "{}-12-31".format(YEAR))
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
flights_tiny_df = flights_tiny_df \
    .withColumn(
        "scheduled_departure_timestamp", 
        F.to_timestamp(
            F.when(
                F.col("scheduled_depature_time") == "24:00:00",
                "00:00:00"
            ).otherwise(F.col("scheduled_depature_time")),
            "HH:mm:ss"
        )
    ) \
    .withColumn(
        "actual_departure_timestamp", 
        F.to_timestamp(
            F.when(
                F.col("actual_departure_time") == "24:00:00",
                "00:00:00"
            ).otherwise(F.col("actual_departure_time")),
            "HH:mm:ss"
        )
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
flights_tiny_df = flights_tiny_df \
    .withColumn("delayed_time",
        F.when(
            F.col("actual_departure_timestamp").cast("long") -
            F.col("scheduled_departure_timestamp").cast("long") > (60*60*12),
            (
                F.col("scheduled_departure_timestamp").cast("long") + (60*60*24) -
                F.col("actual_departure_timestamp").cast("long")
            )/60
        ).when(
            F.col("scheduled_departure_timestamp").cast("long") -
            F.col("actual_departure_timestamp").cast("long") > (60*60*12),
            (
                F.col("actual_departure_timestamp").cast("long") + (60*60*24) -
                F.col("scheduled_departure_timestamp").cast("long")
            )/60
        ).otherwise(
            (F.col("actual_departure_timestamp").cast("long") -
             F.col("scheduled_departure_timestamp").cast("long")
            )/60
        )
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
flights_tiny_df \
    .filter(F.col("actual_departure_time") == "24:00:00") \
    .select("flight_id", "carrier_code", 
#             "scheduled_depature_time", "scheduled_departure_timestamp",
            "actual_departure_time", "actual_departure_timestamp") \
    .show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------+---------------------+--------------------------+
|flight_id|carrier_code|actual_departure_time|actual_departure_timestamp|
+---------+------------+---------------------+--------------------------+
| 15759452|          DL|             24:00:00|       1970-01-01 00:00:00|
| 18640553|          UA|             24:00:00|       1970-01-01 00:00:00|
+---------+------------+---------------------+--------------------------+

In [20]:
flights_tiny_df.select(flight_cols) \
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------+------------+
|carrier_code|tail_number|delayed_time|
+------------+-----------+------------+
|          DL|     N536DA|        20.0|
|          DL|     N629DL|        90.0|
|          DL|     N604DL|         8.0|
|          DL|     N633DL|        11.0|
|          HP|     N906AW|        39.0|
|          HP|     N914AW|         0.0|
|          HP|     N905AW|         4.0|
|          HP|     N181AW|        -1.0|
|          HP|     N183AW|        -4.0|
|          HP|     N632AW|        12.0|
|          HP|     N638AW|        -4.0|
|          HP|     N179AW|         0.0|
|          HP|     N639AW|        32.0|
|          HP|     N185AW|         0.0|
|          HP|     N627AW|         0.0|
|          UA|     N1814U|        48.0|
|          UA|     N345UA|         2.0|
|          UA|     N7254U|        25.0|
|          UA|     N574UA|        17.0|
|          UA|     N556UA|        20.0|
+------------+-----------+------------+
only showing top 20 rows

In [21]:
flights_tiny_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
flights_tiny_df = flights_tiny_df \
    .filter(F.col("actual_departure_timestamp").isNotNull())
flights_tiny_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

54759

In [23]:
avg_flights = flights_tiny_df.select(flight_cols) \
    .filter(F.col("delayed_time") > 0) \
    .groupBy("carrier_code") \
    .agg(
        F.count('delayed_time').alias("numOfDelays"),
        F.mean('delayed_time').alias("avgDelays"),
        F.min('delayed_time').alias("minDelay"),
        F.max('delayed_time').alias("maxDelay")
    ) \
    .withColumn("avgDelays", F.round(F.col("avgDelays"), 2)) \
    .join(airlines_df, 
          flights_tiny_df.carrier_code == airlines_df.carrier_code
    ) \
    .select("name", "numOfDelays", "avgDelays", "minDelay", "maxDelay")
avg_flights.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+---------+--------+--------+
|                name|numOfDelays|avgDelays|minDelay|maxDelay|
+--------------------+-----------+---------+--------+--------+
|     United Airlines|       8975|    21.39|     1.0|   705.0|
|American Airlines...|       6707|   117.17|     1.0|   719.0|
|Northwest Airline...|       2123|    27.23|     1.0|   373.0|
|    Sparrow Aviation|       1086|    28.48|     1.0|   578.0|
|            Tway Air|        590|    35.29|     1.0|   454.0|
|Delta Air Lines Inc.|       6288|     20.4|     1.0|   380.0|
|          US Airways|        823|    18.54|     1.0|   420.0|
|Alaska Airlines Inc.|        219|    29.56|     1.0|   185.0|
|Continental Air L...|       3182|    20.27|     1.0|   315.0|
|Southwest Airline...|       1537|     23.4|     1.0|   247.0|
+--------------------+-----------+---------+--------+--------+

In [24]:
avg_flights.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10

In [25]:
def parseAvgFlights(avg_flights_df):
    results = avg_flights_df.collect()
    results.sort(key=operator.itemgetter(0))
    for i in range(len(results)):
        print("{} \t {} \t {} \t {} \t {}".format(
            results[i][0], 
            results[i][1],
            results[i][2],
            results[i][3],
            results[i][4]
        ), file=sys.stdout)
        
parseAvgFlights(avg_flights)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Alaska Airlines Inc. 	 219 	 29.56 	 1.0 	 185.0
American Airlines Inc. 	 6707 	 117.17 	 1.0 	 719.0
Continental Air Lines Inc. 	 3182 	 20.27 	 1.0 	 315.0
Delta Air Lines Inc. 	 6288 	 20.4 	 1.0 	 380.0
Northwest Airlines Inc. 	 2123 	 27.23 	 1.0 	 373.0
Southwest Airlines Co. 	 1537 	 23.4 	 1.0 	 247.0
Sparrow Aviation 	 1086 	 28.48 	 1.0 	 578.0
Tway Air 	 590 	 35.29 	 1.0 	 454.0
US Airways 	 823 	 18.54 	 1.0 	 420.0
United Airlines 	 8975 	 21.39 	 1.0 	 705.0

# Task 3

In [74]:
airlines_df.select(F.col("country")) \
    .distinct() \
    .count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

82

In [149]:
airlines_df \
    .filter(F.col("country") == "United States") \
    .filter(F.col("name").like("%Tran%")) \
    .sort("name") \
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+-------------+
|carrier_code|                name|      country|
+------------+--------------------+-------------+
|          8C|Air Transport Int...|United States|
|          FL|             AirTran|United States|
|         CTL|Coastal Air Trans...|United States|
|         CTQ|Custom Air Transport|United States|
|         TIA|      Trans Air Inc.|United States|
|         TIE|Trans Internation...|United States|
|         PDA|Trans Southern Ai...|United States|
|          AX|Trans States Airl...|United States|
|         TWA|Trans Western Air...|United States|
|         TWE| Trans World Express|United States|
|         TCO|Trans-Colorado Ai...|United States|
|          T9|Transmeridian Air...|United States|
+------------+--------------------+-------------+

In [125]:
airlines_df.select(F.col("country")) \
    .distinct() \
    .show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|          country|
+-----------------+
|           Russia|
|           Sweden|
|Republic of Korea|
|      Philippines|
|        Singapore|
|         Malaysia|
|             Iraq|
|          Germany|
|         Maldives|
|      Ivory Coast|
+-----------------+
only showing top 10 rows

In [126]:
COUNTRY="United States"
FILE=flights_tiny

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
flights_tiny_df = spark.read.csv(FILE, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
aircraft_col = [
    "manufacturer", 
    "model",
    "modelName",
    "tailnum",
]

'''
QUESTION 1: If we drop the null values for aircraft, then
            what's gonna happen if there is a tailNum but 
            there's no model + manufacturer for it?
            
            Well... there's no info to output for
            aircraft type as well so...
'''

aircraft_modelName = aircraft_df \
    .filter(F.col("manufacturer").isNotNull()) \
    .filter(F.col("model").isNotNull()) \
    .withColumn("modelName", 
                F.regexp_extract(
                    F.col("model"), '\d{3}', 0)
               ) \
    .select(aircraft_col)
aircraft_modelName.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------+---------+-------+
|    manufacturer|    model|modelName|tailnum|
+----------------+---------+---------+-------+
|         EMBRAER|EMB-145XR|      145| N10156|
|AIRBUS INDUSTRIE| A320-214|      320| N102UW|
|          BOEING|  737-3TO|      737| N10323|
|AIRBUS INDUSTRIE| A320-214|      320| N103US|
|          BOEING|  747-422|      747| N104UA|
|AIRBUS INDUSTRIE| A320-214|      320| N104UW|
|         EMBRAER|EMB-145LR|      145| N10575|
|          BOEING|  747-451|      747| N105UA|
|AIRBUS INDUSTRIE| A320-214|      320| N105UW|
|AIRBUS INDUSTRIE| A320-214|      320| N106US|
|          BOEING|  747-422|      747| N107UA|
|AIRBUS INDUSTRIE| A320-214|      320| N107US|
|AIRBUS INDUSTRIE| A320-214|      320| N108UW|
|AIRBUS INDUSTRIE| A320-214|      320| N109UW|
|AIRBUS INDUSTRIE| A320-214|      320| N110UW|
|         EMBRAER|EMB-145XR|      145| N11106|
|         EMBRAER|EMB-145XR|      145| N11107|
|         EMBRAER|EMB-145XR|      145| N11109|
|         EMB

In [129]:
airlines_df.filter(airlines_df["country"] == COUNTRY) \
    .join(flights_tiny_df, airlines_df.carrier_code == flights_tiny_df.carrier_code) \
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+---------------+-------------+---------+------------+-------------+-----------+------+-----------+-----------+-----------------------+----------------------+---------------------+-------------------+--------+
|carrier_code|           name|      country|flight_id|carrier_code|flight_number|flight_date|origin|destination|tail_number|scheduled_depature_time|scheduled_arrival_time|actual_departure_time|actual_arrival_time|distance|
+------------+---------------+-------------+---------+------------+-------------+-----------+------+-----------+-----------+-----------------------+----------------------+---------------------+-------------------+--------+
|          UA|United Airlines|United States|  5017626|          UA|          370| 1994-05-23|   ORD|        PIT|       null|               20:15:00|              22:32:00|             20:17:00|           22:42:00|     412|
|          UA|United Airlines|United States|  5017876|          UA|          376| 1994-05-12|   ORD|        

In [130]:
airlines_all_counts = airlines_df.filter(airlines_df["country"] == COUNTRY) \
    .join(flights_tiny_df, airlines_df.carrier_code == flights_tiny_df.carrier_code) \
    .drop(flights_tiny_df.carrier_code) \
    .filter(F.col("actual_departure_time").isNotNull()) \
    .groupBy("name", "carrier_code", "tail_number") \
    .count() \
    .join(aircraft_modelName, 
        flights_tiny_df.tail_number == aircraft_modelName.tailnum,
        how="left"
    ) \
    .filter(F.col("manufacturer").isNotNull()) \
    .filter(F.col("model").isNotNull()) \
    .groupBy("name", "manufacturer", "model") \
    .sum("count") \
    .withColumnRenamed("sum(count)", "numFlights") \
    .cache()
airlines_all_counts.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+---------+----------+
|                name|     manufacturer|    model|numFlights|
+--------------------+-----------------+---------+----------+
|Delta Air Lines Inc.|           BOEING|  767-332|      6489|
|Delta Air Lines Inc.|MCDONNELL DOUGLAS| MD-90-30|      2270|
|Alaska Airlines Inc.|           BOEING|  737-8FH|        30|
|Southwest Airline...|           CESSNA|    T337G|         8|
|Northwest Airline...|           AIRBUS| A319-114|      2240|
|Continental Air L...|           BOEING|  737-824|      9467|
|     United Airlines| AIRBUS INDUSTRIE| A320-232|     15134|
|          US Airways|           AIRBUS| A320-232|       724|
|     United Airlines|           AIRBUS| A319-131|       702|
|   PSA Airlines Inc.|            PIPER|PA-28-180|        34|
|American Airlines...|           BOEING|  757-223|      3708|
|American Airlines...|           CESSNA|     421C|       535|
|Delta Air Lines Inc.|           BOEING|777-232LR|         2|
|Alaska 

In [131]:
airlines_all_counts.groupBy("name") \
    .count() \
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|                name|count|
+--------------------+-----+
|    Independence Air|    1|
|        JetSuite Air|    6|
|Skywest Airlines ...|    6|
|American Eagle Ai...|    6|
|     United Airlines|   11|
|              Comair|    5|
|   PSA Airlines Inc.|    5|
|Frontier Airlines...|    6|
|Southwest Airline...|   22|
|Continental Air L...|   13|
|Northwest Airline...|   16|
|     JetBlue Airways|    3|
|          US Airways|   24|
|Hawaiian Airlines...|    4|
|             AirTran|    3|
|Atlantic Southeas...|    5|
|Alaska Airlines Inc.|    9|
|Delta Air Lines Inc.|   17|
|  Mesa Airlines Inc.|    6|
|American Airlines...|   43|
+--------------------+-----+

We don't groupBy with "modelName" for now because some of the names will disappear

`.groupBy("name", "manufacturer", "model", "modelName") \`

In [132]:
window = Window.partitionBy(
    airlines_all_counts["name"]).orderBy(
    airlines_all_counts["numFlights"].desc()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
'''
QUESTION 2: If we drop the null values for manufacturer and
            and model now, then we won't incorrectly count
            the number of flights since "null" was grouped
            together in the previous function.
'''

airlines_model_ranking = airlines_all_counts \
    .select("*", F.rank().over(window).alias("rank")) \
    .filter(F.col("rank") <=5) \
    .sort(["name", "rank"], ascending=True)
airlines_model_ranking.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+---------------+----------+----+
|                name|        manufacturer|          model|numFlights|rank|
+--------------------+--------------------+---------------+----------+----+
|             AirTran|              BOEING|        717-200|     11225|   1|
|             AirTran|              BOEING|        737-76N|      1858|   2|
|             AirTran|              BOEING|        737-7BD|      1674|   3|
|Alaska Airlines Inc.|              BOEING|        737-4Q8|      1380|   1|
|Alaska Airlines Inc.|              BOEING|        737-790|       859|   2|
|Alaska Airlines Inc.|              BOEING|        737-490|       814|   3|
|Alaska Airlines Inc.|   MCDONNELL DOUGLAS| DC-9-83(MD-83)|       767|   4|
|Alaska Airlines Inc.|              BOEING|        737-990|       441|   5|
|American Airlines...|   MCDONNELL DOUGLAS| DC-9-82(MD-82)|     37353|   1|
|American Airlines...|   MCDONNELL DOUGLAS| DC-9-83(MD-83)|      7019|   2|
|American Ai

In [134]:
airlines_model_ranking.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

91

In [135]:
def parsePopularModels(airlines_model_ranking):
    results = airlines_model_ranking.collect()
    
    if len(results) == 0:
        return

    results.sort(key=operator.itemgetter(0, 4))
    currAirline = results[0][0]
      
    i = 0
    hasPrintOnce = False
    aircraftTypeString = ""
    while i < len(results):
        if currAirline == results[i][0]:
            aircraftTypeString += "{} {}, ".format(results[i][1], results[i][2])
            i += 1
        else:
            hasPrintOnce = True
            print(
                "{} \t [{}]".format(
                    currAirline, 
                    aircraftTypeString[:-2]
                ), file=sys.stdout
            )
            aircraftTypeString = ""
            currAirline = results[i][0]
    
    if not hasPrintOnce:
         print(
            "{} \t [{}]".format(
                currAirline, 
                aircraftTypeString[:-2]
            ), file=sys.stdout
        )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
parsePopularModels(airlines_model_ranking)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AirTran 	 [BOEING 717-200, BOEING 737-76N, BOEING 737-7BD]
Alaska Airlines Inc. 	 [BOEING 737-4Q8, BOEING 737-790, BOEING 737-490, MCDONNELL DOUGLAS DC-9-83(MD-83), BOEING 737-990]
American Airlines Inc. 	 [MCDONNELL DOUGLAS DC-9-82(MD-82), MCDONNELL DOUGLAS DC-9-83(MD-83), BOEING 757-223, BOEING 767-223, BOEING 767-323]
American Eagle Airlines Inc. 	 [EMBRAER EMB-145LR, EMBRAER EMB-135KL, BOMBARDIER INC CL-600-2C10, SAAB-SCANIA SAAB 340B, EMBRAER EMB-135LR]
Atlantic Southeast Airlines 	 [BOMBARDIER INC CL-600-2B19, BOMBARDIER INC CL-600-2C10, CANADAIR CL-600-2B19, AEROSPATIALE/ALENIA ATR-72-212, AEROSPATIALE ATR 72-212]
Comair 	 [BOMBARDIER INC CL-600-2B19, CANADAIR CL-600-2B19, BOMBARDIER INC CL-600-2C10, BOMBARDIER INC CL600-2D24, PIPER PA-28-180]
Continental Air Lines Inc. 	 [BOEING 737-524, BOEING 737-3TO, BOEING 737-824, BOEING 757-224, BOEING 737-724]
Delta Air Lines Inc. 	 [MCDONNELL DOUGLAS AIRCRAFT CO MD-88, BOEING 757-232, BOEING 767-332, BOEING 737-832, MCDONNELL DOUGLAS CO